Grupo 12

Francisco de Borja Lozano del Moral

Manuel Ortega Salvador

# Código para el problema 1 - Linterna

## Definición del problema
El problema es expresado como una tupla, donde:

    [0] == vector de personas, donde cada persona contiene su velocidad y si debe cruzar,
        ej. ((10,0), (30,0), (60,0), (80,0), (120,0)) nadie ha cruzado aún

    [1] == la linterna está en el origen (izquierda)

    [2] == cantidad de tiempo que queda

Al expresarlo como una tupla, tiene hash y se puede comparar a otras tuplas. Esto hace que se pueda meter en el
bin. heap más adelante.

## Elección del algoritmo

Los algoritmos probados son:

    1. A* con varias heurísticas: astar_search  
    2. Dijkstra's (árbol): breadth_first_tree_search

## Complejidad del algoritmo

Los algoritmos tienen siempre como complejidad en caso peor o mejor $O(n) = D(n)^{B(n)}$ donde D es la profundidad de la solución y B es el número de ramas por nodo.

Para ver el numero de acciones posibles, sea $i$ la cantidad de personas que quedan por cruzar y sea $n-i$ los que ya han cruzado.

Entonces, hay en la ida tantas posibles acciones como parejas posibles si hay más de dos personas que quedan por cruzar y una en caso contrario.

Por esto, en la ida hay ${i \choose 2} = i(i-1)\frac{1}{2} = \frac{1}{2} (i^2 - i)$ si $i \geq 2$ y 1 en caso contrario. 

En la vuelta hay tantas posibles acciones personas quedan a la vuelta, lo limitamos a que solo una persona puede volver porque nunca vale la pena que vuelvan dos personas con la linterna.

Por esto, en la vuelta hay $n-i$ posibles acciones.

Y por lo tanto, en una ida y vuelta hay $b = {i \choose 2}(n-i-2)$ acciones posibles, tras lo cual $i$ incrementa en 1.

Por ello tenemos que hay en una ida y vuelta ${i \choose 2}(n-i-2) = \frac{1}{2}i(i-1)(n-i-2)$ ramas.

Para concluir, como va a haber en torno a $n$ viajes de ida y vuelta, tenemos que se van a visitar en el peor caso
$$C(n) =\prod_{i=1}^n \frac{1}{2}i(i-1)(n-i-2))$$ nodos.

$$C(n) \subset O(\prod_{i=1}^n \frac{1}{2}i(i-1)(n-i-2)) \subset O( \frac{1}{2^n}\prod_{i=2}^n i^2n - i^3 - 2i^2 + i^2- in +2i) \subset O(\frac{1}{2^n}\prod_{i=2}^n i^2n - i^3) \subset O((\frac{n}{2})^n (n^3)^n) \subset O((\frac{1}{2})^n (n^3)^n) \subset O((\frac{n^3}{2})^n)\subset O(0.5^n n^{3n})\subset O(n^{3n})$$

Este resultado parece correcto, ya que si se cambia la estructura del grafo y la representacion del problema para tratar en vez de trayectos, idas y vueltas, en cada nivel del arbol se abren $n^3$ ramas y tendría una profundidad lineal respecto a $n$.

In [2]:
from search import *

In [3]:
class FlashAndBridge(Problem):
    """
    El problema es expresado como un tuple, donde:
    [0] == vector de personas, donde cada persona contiene su velocidad y si debe cruzar,
        ej. ((10,0), (30,0), (60,0), (80,0), (120,0)) nadie ha cruzado aún
    [1] == la linterna está en el origen (izquierda)
    [2] == cantidad de tiempo que queda

    Al expresarlo como una tupla, tiene hash y se puede comparar a otras tuplas. Esto hace que se pueda meter en el
    bin. heap más adelante.
    """
    def __init__(self, speeds, time):
        self.initial = (
            tuple([(speed, 0) for speed in speeds]),
            True,
            time
        )

    def actions(self, state):
        """
        Devuelve la cantidad de acciones posibles en este turno. No admite como posibles acciones que dejan
        tiempo negativo.

        Las acciones van a estar expresadas como tuplas de personas que cruzan, que pueden tener tamaño 1 o 2.
        """
        actions = []
        time = state[2]
        light_left = state[1]
        people = state[0]

        if light_left:
            """
            Si la linterna está en el lado izquierdo puede cruzar una o dos personas.
            """
            valid_indexes = [i for i in range(len(people)) if people[i][0] <= time and people[i][1] == 0]
            N = len(valid_indexes)
            if len(valid_indexes) == 1:
                actions.append((valid_indexes[0],))
            else:
                for i in range(N):
                    for j in range(i + 1, N):
                        actions.append((valid_indexes[i], valid_indexes[j]))
        else:
            """
            Si la linterna está en el lado derecho solo conviene que cruze de vuelta una persona
            """
            actions = [(i,) for i in range(len(people)) if people[i][0] <= time and people[i][1] == 1]
        return actions

    def result(self, state, action):
        """
        Realiza la acción. Cruza o una o dos personas de un lado al otro.
        """
        people = list(state[0])
        light = state[1]
        time = state[2]

        if light and len(action) > 1:
            x, y = action
            xs, ys = people[x][0], people[y][0]
            time -= max(xs, ys)
            people[x] = people[x][0], 1
            people[y] = people[y][0], 1
        else:
            action = action[0]
            time -= people[action][0]
            people[action] = people[action][0], 1 - people[action][1]

        return tuple(people), not light, time

    def goal_test(self, state):
        """
        Devuelve verdadero si todos han cruzado.
        """
        for speed, position in state[0]:
            if position == 0:
                return False
        return True


class Problema_con_Analizados(Problem):
   """Es un problema que se comporta exactamente igual que el que recibe al
      inicializarse, y además incorpora unos atributos nuevos para almacenar el
      número de nodos analizados durante la búsqueda. De esta manera, no
      tenemos que modificar el código del algoritmo de búsqueda."""

   def __init__(self, problem):
       self.initial = problem.initial
       self.problem = problem
       self.analizados = 0

   def actions(self, estado):
       return self.problem.actions(estado)

   def result(self, estado, accion):
       return self.problem.result(estado, accion)

   def goal_test(self, estado):
       self.analizados += 1
       b = self.problem.goal_test(estado)
#        if b:
#            print("\nTiempo: {}/{}".format(self.initial[2]-estado[2], self.initial[2]))
       return b

   def coste_de_aplicar_accion(self, estado, accion):
       return self.problem.coste_de_aplicar_accion(estado,accion)


def resuelve_problema(estado_inicial, algoritmo, h=None, printing=True):
        p=Problema_con_Analizados(FlashAndBridge(*estado_inicial))
        if h:
            sol= algoritmo(p,h).solution()
        else:
            sol= algoritmo(p).solution()
        
        if printing:
            print("Solución: {0}".format(sol))
            print("Algoritmo: {0}".format(algoritmo.__name__))
            if h:
                print("Heurística: {0}".format(h.__name__))
            else:
                pass
            print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol), p.analizados))




## Definimos ahora alguna heurística
Como heurística básica, usamos la cantidad de personas que quedan por cruzar.

Como segunda heurística, usamos la cantidad de tiempo que llevamos, esto hace que el problema se vuelva Dijkstra's y encuentre la solución óptima.

estimation_h es una estimación simple tiempo que va a llevar cruzar a todo el mundo.

En h2, suponemos que la linterna no se queda sin batería, por lo que el tiempo que tarda cada persona en cruzar no importa.

In [4]:
def basic_h(node):
    state = node.state
    total = 0
    for (speed, pos) in state[0]:
        total += 1-pos
    total += 1 if state[1] else 0
    return total


def time_h(node):
    return node.state[2]


def estimation_h(node):
    state = node.state
    total = state[2]
    left = 0
    q = 0
    min_right = 0

    for (speed, pos) in state[0]:
        if pos == 0:
            left += speed
            q += 1
        else:
            if min_right == 0 or speed < min_right:
                min_right = speed
    left = left/q if q > 0 else 0
    return total - left - min_right

def h2(node):
    state = node.state
    num = 0
    for (speed, pos) in state[0]:
        num += 1-pos
    return 2*(num-state[2])-state[2]

In [5]:
inicial = ((10, 30, 60, 80, 120), 5 * 60)
problema = FlashAndBridge(*inicial)

In [6]:
%%timeit
astar_search(problema, time_h)

3.81 ms ± 665 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
resuelve_problema(inicial, astar_search, time_h)

Solución: [(0, 2), (0,), (0, 1), (1,), (3, 4), (0,), (0, 1)]
Algoritmo: astar_search
Heurística: time_h
Longitud de la solución: 7. Nodos analizados: 249


In [8]:
%%timeit
astar_search(problema, basic_h)

14.9 ms ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
resuelve_problema(inicial, astar_search, basic_h)

Solución: [(0, 1), (0,), (3, 4), (1,), (0, 2), (0,), (0, 1)]
Algoritmo: astar_search
Heurística: basic_h
Longitud de la solución: 7. Nodos analizados: 218


In [61]:
%%timeit
astar_search(problema, estimation_h)

3.2 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
resuelve_problema(inicial, astar_search, estimation_h)

Solución: [(0, 2), (0,), (0, 1), (1,), (3, 4), (0,), (0, 1)]
Algoritmo: astar_search
Heurística: estimation_h
Longitud de la solución: 7. Nodos analizados: 249


In [63]:
%%timeit
astar_search(problema, h2)

22.8 ms ± 887 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
resuelve_problema(inicial, astar_search, h2)

Solución: [(0, 1), (0,), (3, 4), (1,), (0, 1), (0,), (0, 2)]
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 7. Nodos analizados: 242


In [65]:
%%timeit
uniform_cost_search(problema)

16.6 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
resuelve_problema(inicial, uniform_cost_search)

Solución: [(0, 2), (0,), (0, 1), (0,), (3, 4), (1,), (0, 1)]
Algoritmo: uniform_cost_search
Longitud de la solución: 7. Nodos analizados: 274


In [67]:
%%timeit
breadth_first_graph_search(problema)

7.31 ms ± 477 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [68]:
resuelve_problema(inicial, breadth_first_graph_search)

Solución: [(0, 1), (0,), (0, 2), (0,), (3, 4), (1,), (0, 1)]
Algoritmo: breadth_first_graph_search
Longitud de la solución: 7. Nodos analizados: 271


In [69]:
%%timeit

depth_first_graph_search(problema)

2.09 ms ± 49.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [70]:
resuelve_problema(inicial, depth_first_graph_search)

Solución: [(0, 2), (0,), (0, 1), (1,), (3, 4), (0,), (0, 1)]
Algoritmo: depth_first_graph_search
Longitud de la solución: 7. Nodos analizados: 249


In [71]:
%%timeit
depth_first_tree_search(problema)

5.18 ms ± 484 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
resuelve_problema(inicial, depth_first_tree_search)

Solución: [(0, 2), (0,), (0, 1), (1,), (3, 4), (0,), (0, 1)]
Algoritmo: depth_first_tree_search
Longitud de la solución: 7. Nodos analizados: 1250


In [73]:
%%timeit
breadth_first_tree_search(problema)

4.61 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [74]:
resuelve_problema(inicial, breadth_first_tree_search)

Solución: [(0, 1), (0,), (0, 2), (0,), (3, 4), (1,), (0, 1)]
Algoritmo: breadth_first_tree_search
Longitud de la solución: 7. Nodos analizados: 994


## Tabla de resultados

| Algorithm 	| H 	| Nodes 	| Time | Solution 	|  	
|:-:	|:-:	|:-:	|:-:	|:-:	|
| A* 	| time_h 	| 249 	|   3.26 ms ± 244 µs	|  [(0, 2), (0,), (0, 1), (1,), (3, 4), (0,), (0, 1)]	|
| A* 	| estimation_h 	| 249 	| 3.2 ms ± 119 µs	| [(0, 2), (0,), (0, 1), (1,), (3, 4), (0,), (0, 1)] 	|
| A* 	| basic_h 	| 218 | 13.1 ms ± 365 µs 	| [(0, 1), (0,), (3, 4), (1,), (0, 2), (0,), (0, 1)] 	|
| A* 	| h2 	| 242 	| 22.8 ms ± 887 | [(0, 1), (0,), (3, 4), (1,), (0, 1), (0,), (0, 2)]  	|
| BFTS 	| - 	| 994 	| 4.61 ms ± 193 µs 	| [(0, 1), (0,), (0, 2), (0,), (3, 4), (1,), (0, 1)] 	|
| BFGS 	| - 	| 271 	| 7.31 ms ± 477 µs  |  [(0, 1), (0,), (0, 2), (0,), (3, 4), (1,), (0, 1)]	|
| DFTS 	| - 	| 1250 	| 5.18 ms ± 484 µs 	|  [(0, 2), (0,), (0, 1), (1,), (3, 4), (0,), (0, 1)]	|
| DFGS	| -	    | 249 	| 2.09 ms ± 49.8 µs	| [(0, 2), (0,), (0, 1), (1,), (3, 4), (0,), (0, 1)] 	|


Como podemos observar, depth first graph search es el algoritmo más rápido en este caso, pero apenas es mejor que A* usando time_h o estimation_h. Podemos ahora compararlos en otros casos más complejos:

In [75]:
inicial = ((10, 30, 60, 80, 120, 10, 50), 6 * 60)
problema = FlashAndBridge(*inicial)

In [76]:
%%timeit
astar_search(problema, time_h)

16.6 ms ± 679 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [77]:
resuelve_problema(inicial, astar_search, time_h)

Solución: [(4, 5), (5,), (3, 5), (5,), (1, 5), (5,), (0, 5), (0,), (2, 6), (5,), (0, 5)]
Algoritmo: astar_search
Heurística: time_h
Longitud de la solución: 11. Nodos analizados: 603


In [80]:
%%timeit
astar_search(problema, estimation_h)

37.7 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [81]:
resuelve_problema(inicial, astar_search, estimation_h)

Solución: [(1, 4), (1,), (0, 5), (0,), (1, 3), (5,), (0, 5), (0,), (2, 6), (5,), (0, 5)]
Algoritmo: astar_search
Heurística: estimation_h
Longitud de la solución: 11. Nodos analizados: 1215


In [82]:
%%timeit

depth_first_graph_search(problema)

18.4 ms ± 513 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [83]:
resuelve_problema(inicial, depth_first_graph_search)

Solución: [(5, 6), (5,), (4, 5), (5,), (1, 5), (5,), (0, 5), (5,), (2, 3), (0,), (0, 5)]
Algoritmo: depth_first_graph_search
Longitud de la solución: 11. Nodos analizados: 1038


In [84]:
%%timeit
depth_first_tree_search(problema)

116 ms ± 4.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [85]:
resuelve_problema(inicial, depth_first_tree_search)

Solución: [(5, 6), (5,), (4, 5), (5,), (1, 5), (5,), (0, 5), (5,), (2, 3), (0,), (0, 5)]
Algoritmo: depth_first_tree_search
Longitud de la solución: 11. Nodos analizados: 30835


Como se podía preveer, A* supera a DFS cuando el número de ramas que tiene que explorar DFS aumenta. Otra cosa que conviene comentar es que no solo es A* más rápido con time_h que las otras opciones, sino que además devuelve la solución que usa el menor tiempo posible.